In [3]:
# GET DATA
import os
import matplotlib.pyplot as plt

!rm -rf "/content/knowledge-graph-question-answering-imdb1000"

!git clone https://github.com/shariqfz/knowledge-graph-question-answering-imdb1000.git

data_path = '/content/knowledge-graph-question-answering-imdb1000/data/IMDB-Movie-Data.csv'

Cloning into 'knowledge-graph-question-answering-imdb1000'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 16 (delta 3), reused 4 (delta 0), pack-reused 0
Receiving objects: 100% (16/16), 142.58 KiB | 1.85 MiB/s, done.
Resolving deltas: 100% (3/3), done.


In [1]:
!pip install Node2vec

In [4]:
import pandas as pd
import networkx as nx
from node2vec import Node2Vec

# Load the data
df = pd.read_csv(data_path)

# Create a directed graph
G = nx.DiGraph()

# Add nodes and edges based on your data
for index, row in df.iterrows():
    movie = row['Title']
    G.add_node(movie, title=movie, genre=row['Genre'], director=row['Director'], actors=row['Actors'], year=row['Year'])

    # Add edges between movie and genresfor index, row in df.iterrows():
    movie = row['Title']
    G.add_node(movie, title=movie, genre=row['Genre'], director=row['Director'], actors=row['Actors'], year=row['Year'])

    # Add edges between movie and genres
    for genre in row['Genre'].split(','):
        if not G.has_node(genre):
            G.add_node(genre)
        G.add_edge(movie, genre, relation='belongs to')

    # Add edges between movie and director
    director = row['Director']
    if not G.has_node(director):
        G.add_node(director)
    G.add_edge(movie, director, relation='directed by')

    # Add edges between movie and actors
    for actor in row['Actors'].split(','):
        if not G.has_node(actor):
            G.add_node(actor)
        G.add_edge(actor, movie, relation='acted in')

    # Add edge between movie and year
    year = row['Year']
    if not G.has_node(year):
        G.add_node(year)
    G.add_edge(movie, year, relation='released in')

# Precompute probabilities and generate walks
graph_embedding_dim = 768
node2vec = Node2Vec(G, dimensions=graph_embedding_dim, walk_length=10, num_walks=20, workers=4)

# Embed nodes
n2v_model = node2vec.fit(window=10, min_count=1, batch_words=4)

# Get embeddings for all nodes in your graph
embeddings = n2v_model.wv.vectors

# Create a mapping of node names to integers
node_mapping = {node: i for i, node in enumerate(n2v_model.wv.index_to_key)}
index_to_node = {i: node for i, node in enumerate(n2v_model.wv.index_to_key)}
# Now, `node_mapping` is a dictionary that maps node names to their corresponding indices in the `embeddings` array
print(node_mapping)

Computing transition probabilities:   0%|          | 0/4046 [00:00<?, ?it/s]

{'Drama': 0, '2016': 1, 'Comedy': 2, 'Action': 3, 'Adventure': 4, 'Thriller': 5, 'Horror': 6, 'Romance': 7, 'Crime': 8, '2015': 9, 'Mystery': 10, 'Sci-Fi': 11, '2014': 12, 'Fantasy': 13, '2013': 14, 'Biography': 15, '2012': 16, '2011': 17, '2008': 18, '2010': 19, '2007': 20, '2009': 21, 'Animation': 22, 'Family': 23, '2006': 24, 'History': 25, 'Sport': 26, 'Music': 27, 'Gran Torino': 28, 'War': 29, 'Clint Eastwood': 30, 'To Rome with Love': 31, 'The Host': 32, 'Tracktown': 33, '5- 25- 77': 34, 'Woody Allen': 35, 'Chef': 36, 'M. Night Shyamalan': 37, 'Popstar: Never Stop Never Stopping': 38, 'Lars von Trier': 39, 'Mike Flanagan': 40, 'Paul W.S. Anderson': 41, 'Western': 42, 'Blair Witch': 43, 'I, Daniel Blake': 44, 'The Fast and the Furious: Tokyo Drift': 45, 'En man som heter Ove': 46, 'Dragonball Evolution': 47, 'Wrecker': 48, 'Hunger': 49, 'Jagten': 50, 'Viral': 51, 'Hostel: Part II': 52, 'The Neighbor': 53, 'Incendies': 54, 'I.T.': 55, 'I Origins': 56, 'Bacalaureat': 57, 'City of Ti

In [6]:
# Let's assume `node_value` is the value of the node for which you want to get the embedding
node_value = "2016"

# Get the embedding vector for the node
node_embedding = n2v_model.wv[node_value]

print(len(node_embedding))


768


In [33]:
print(index_to_node[int(most_relevant_node.numpy())])

The Perks of Being a Wallflower


In [6]:
import pandas as pd

train_data_path = "/content/knowledge-graph-question-answering-imdb1000/data/dataset.csv"
df = pd.read_csv(train_data_path)
answer_embeddings = []
questions = []
for index, row in df.iterrows():
  node_value = row['answer']
  if node_value in n2v_model.wv:
    # print(node_value)
    answer_embeddings.append(n2v_model.wv[node_value])
    questions.append(row.question)

print(questions[0])
print(len(questions))

In which year was the movie "Interstellar" with Matthew McConaughey released?
159


In [ ]:
num_epochs = 3
import numpy as np
from transformers import BertTokenizer
import torch

# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

text_queries = questions
# Tokenize the queries
inputs = tokenizer(text_queries, return_tensors='pt', padding=True, truncation=True)

# Convert node embeddings to torch tensors
# node_embeddings = torch.tensor(embeddings)
answer_embeddings = torch.tensor(np.array(answer_embeddings))

from transformers import BertModel
import torch.nn as nn

class FineTunedBert(nn.Module):
    def __init__(self, bert_model_name, node_embedding_dim):
        super(FineTunedBert, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.fc = nn.Linear(self.bert.config.hidden_size, node_embedding_dim)

    def forward(self, input_ids, attention_mask, token_type_ids=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        out = self.fc(outputs.pooler_output)
        return out


from torch.optim import AdamW

# Initialize the model
model = FineTunedBert('bert-base-uncased', graph_embedding_dim)

# Define the optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Train the model
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(**inputs)
    loss = 1 - torch.nn.functional.cosine_similarity(outputs, answer_embeddings)
    # Aggregate the loss into a scalar
    loss = loss.mean()
    loss.backward()
    optimizer.step()


# Put the model in evaluation mode
model.eval()

# Get the query
query = "Name  movie whose title is Interstellar"

# Prepare the query
inputs = tokenizer(query, return_tensors='pt')

# Get the output from the model
with torch.no_grad():
    output = model(**inputs)

# print(output)

In [18]:
# Get the query
query = "Name movie whose title is Interstellar"

# Prepare the query
inputs = tokenizer(query, return_tensors='pt')

# Get the output from the model
with torch.no_grad():
    output = model(**inputs)

In [19]:
output_vec = outputs[0]
print(len(outputs), len(outputs[0]))

2 1


In [20]:
from torch.nn.functional import cosine_similarity

# transformed_bert_embedding = transform(bert_embedding)
graph_embeddings_tensor = torch.from_numpy(embeddings)

# Calculate the cosine similarity
similarities = cosine_similarity(output_vec, graph_embeddings_tensor, dim=-1)

# Find the most relevant node
most_relevant_node = torch.argmax(similarities)

RuntimeError: ignored

In [21]:
print(most_relevant_node.numpy())
print(index_to_node[int(most_relevant_node.numpy())])

24
2006


In [17]:
from transformers import BertModel, BertTokenizer
import torch
import torch.nn as nn
from torch.optim import AdamW

# Initialize the tokenizer and the model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Add a new layer for fine-tuning
model.add_module('fc', nn.Linear(model.config.hidden_size, graph_embedding_dim))

# Define the optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Train the model
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(**inputs)
    # Get the last hidden state
    last_hidden_state = outputs.last_hidden_state
    # Apply the new layer
    out = model.fc(last_hidden_state)
    loss = 1 - torch.nn.functional.cosine_similarity(out, answer_embeddings)
    # Aggregate the loss into a scalar
    loss = loss.mean()
    loss.backward()
    optimizer.step()

# Put the model in evaluation mode
model.eval()

# Get the query
query = "Name  movie whose title is Interstellar"

# Prepare the query
inputs = tokenizer(query, return_tensors='pt')

# Get the output from the model
with torch.no_grad():
    output = model(**inputs)
    # Apply the new layer
    output = model.fc(output.last_hidden_state)


RuntimeError: ignored

In [16]:
import torch
from torch.nn import Linear
from torch.nn.functional import cosine_similarity
from transformers import BertModel, BertTokenizer

# Check if GPU is available and if not, use CPU
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load pre-trained model and tokenizer
model = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Define the dimension of your graph embeddings
graph_embeddings = embeddings
# Add a linear transformation layer to the model
transform = Linear(model.config.hidden_size, graph_embedding_dim)

# Assume `graph_embeddings` is a 2D tensor where each row is the embedding of a node
# graph_embeddings = ...

# Encode text query
input_ids = tokenizer.encode("Name a 2014 movie whose title is Interstellar", add_special_tokens=True)
input_ids = torch.tensor(input_ids).unsqueeze(0)

# Generate BERT embeddings
with torch.no_grad():
    last_hidden_states = model(input_ids)[0]

# Take the embedding of the first token (i.e., [CLS]) as the embedding of the whole sentence
bert_embedding = last_hidden_states[0, 0, :]

# Transform the BERT embedding to have the same dimension as the graph embeddings
transformed_bert_embedding = transform(bert_embedding)
graph_embeddings_tensor = torch.from_numpy(graph_embeddings)

# Ensure both tensors have the same number of dimensions
if len(transformed_bert_embedding.shape) < len(graph_embeddings_tensor.shape):
    transformed_bert_embedding = transformed_bert_embedding.unsqueeze(0)
elif len(transformed_bert_embedding.shape) > len(graph_embeddings_tensor.shape):
    graph_embeddings_tensor = graph_embeddings_tensor.unsqueeze(0)

# Calculate the cosine similarity
similarities = cosine_similarity(transformed_bert_embedding, graph_embeddings_tensor)

# Find the most relevant node
# most_relevant_node = torch.argmax(similarities)
# print(most_relevant_node.numpy())

# Find the top k most relevant nodes
k = 10
top_k_most_relevant_nodes = torch.topk(similarities, k)

# for t in top_k_most_relevant_nodes:
#   print(t)

node_names = list(G.nodes)

# Get the indices of the top k most relevant nodes
top_k_indices = top_k_most_relevant_nodes.indices

# Get the node names
top_k_node_names = [node_names[i] for i in top_k_indices]

print(top_k_node_names)

[' Angelo Josue Lozano Corzo', ' Seth MacFarlane', 'Gone Girl', ' Julian Barratt', 'Sausage Party', 'Garth Davis', 'Rogue One', ' Scarlett Johansson', 'Drama', ' Iain Glen']


In [ ]:
i = 0  # replace 0 with the actual index

# Get the node value
node_value = index_to_node[i]

print(node_value)